게시판만들기

- 글목록, 글쓰기, 수정, 삭제
- 검색 기능
- 페이지 나누기
- 댓글 쓰기, 댓글 목록
- 파일 업로드, 다운로드

새 프로젝트 만들기: myweb3

앱 생성
```
django-admin startproject config .

python manage.py startapp board
```

### config/settings.py
```config/settings.py

# 날짜 포맷 변경을 위한 모듈 로딩
from django.conf.locale.ko import formats as ko_formats

# 날짜 포맷 설정
ko_formats.DATETIME_FORMAT = 'Y-m-d G:i:s'

INSTALLED_APPS = [
    ...
    'board',
]

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.mysql',
        'NAME': 'pyweb', # DB
        'USER': 'web', # id
        'PASSWORD': '1234', # password
        'HOST': 'localhost', # host
        'PORT': '3306', # port
    }
}

LANGUAGE_CODE = 'ko'

TIME_ZONE = 'Asia/Seoul'
```

#### board/models.py
```board/models.py
from django.db import models
from datetime import datetime

# Create your models here.
class Board(models.Model):
    idx = models.AutoField(primary_key=True)
    writer = models.CharField(null=False, max_length=50)
    title = models.CharField(null=False, max_length=120)
    hit = models.IntegerField(default=0)   # 조회수
    content = models.TextField(null=False)  # 본문
    post_date = models.DateTimeField(default=datetime.now, blank=True)
    filename = models.CharField(null=True, blank=True, default="", max_length=500)
    filesize = models.IntegerField(default=0)
    down = models.IntegerField(default=0)

    def hit_up(self):
        self.hit += 1
    def down_up(self):
        self.down += 1

class Comment(models.Model):
    idx = models.AutoField(primary_key=True)
    board_idx = models.IntegerField(null=False)
    writer = models.CharField(null=False, max_length=50)
    content = models.TextField(null=False)
    post_date = models.DateTimeField(default=datetime.now, blank=True)
```

#### board/admin.py
``` board/admin.py
from django.contrib import admin
from board.models import Board
# Register your models here.

admin.site.register(Board)
#관리자사이트 등록  테이블명

```

#### 변경사항 반영
```
python manage.py makemigrations

python manage.py migrate
```

#### HeidSQL
```
USE pyweb;

SHOW TABLES;

DESC board_board;
DESC board_comment;

```

#### 서버 켜고
```
python manage.py runserver 서버 켜고, 

/admin, 관리자로그인(admin/1234)

게시판 작성
```

#### HeidSQL
```
SELECT * FROM board_board;

```

### mysql 스크립트   (HeidiSQL)

#### 데이터베이스 생성
```
create database pyweb;
```

#### 페이지나누기를 위한 저장프로시저
```
USE pyweb;

DELIMITER $$
DROP PROCEDURE if EXISTS loopInsert$$

CREATE PROCEDURE loopInsert()
BEGIN 
DECLARE i int DEFAULT 1;
DELETE FROM board_board;
while i<=991 do
INSERT INTO board_board(IDX, writer, TITLE, CONTENT, hit, post_date, filesize, down)
VALUES (i, CONCAT('kim', i), CONCAT('제목',i), CONCAT('내용',i), 0, NOW(), 0,0);
SET i = i+1;
END while;
END$$

DELIMITER $$

CALL LoopInsert 

select * from board_board;
```

#### config/urls.py
```
from django.contrib import admin
from django.urls import path
from board import views

urlpatterns = [
    path('admin', admin.site.urls),
    
    # board
    path('', views.list),
    path('write', views.write),
    path('insert', views.insert),
    path('detail', views.detail),
    path('update', views.update),
    path('delete', views.delete),
    path('download', views.download),
    path('reply_insert', views.reply_insert)    
]

```

#### board/views.py
``` board/views.py

from django.shortcuts import render,redirect
from django.db.models import Q
from django.http import HttpResponse, HttpResponseRedirect
from urllib.parse import quote
import math
import os
from board.models import Board, Comment

UPLOAD_DIR = 'c:/workspace/myweb3/upload/'

def list(request):
    try:
        search_option = request.POST["search_option"]
    except:
        search_option = "writer"

    try:
        search = request.POST["search"]
    except:
        search = ""

    boardCount=Board.objects.count()

    try:
        start = int(request.GET['start'])
    except:
        start = 0

    page_size = 10  # 페이지당 게시물수
    page_list_size = 10  # 한 화면에 표시할 페이지의 갯수
    end = start + page_size
    total_page = math.ceil(boardCount / page_size)
    current_page = math.ceil((start + 1) / page_size)
    start_page = math.floor((current_page - 1) / page_list_size) * page_list_size + 1
    end_page = start_page + page_list_size - 1
    if total_page < end_page:
        end_page = total_page
    if start_page >= page_list_size:
        prev_list = (start_page - 2) * page_size
    else:
        prev_list = 0
    if total_page > end_page:
        next_list = end_page * page_size
    else:
        next_list = 0

    if search_option == "all":
        boardList = Board.objects.filter( Q(writer__contains=search) | Q(title__contains=search) | Q(content__contains=search)).order_by("-idx")[start:end]
    elif search_option == "writer":
        boardList = Board.objects.filter(writer__contains=search).order_by("-idx")[start:end]
    elif search_option == "title":
        boardList = Board.objects.filter(title__contains=search).order_by("-idx")[start:end]
    elif search_option == "content":
        boardList = Board.objects.filter(content__contains=search).order_by("-idx")[start:end]

    links = []
    for i in range(start_page, end_page + 1):
        page = (i - 1) * page_size
        links.append("<a href='?start=" + str(page) + "'>" + str(i) + "</a>")

    return render(request, "list.html", {"boardList": boardList, "search_option": search_option,"search": search, "range": range(start_page - 1, end_page), "start_page": start_page,"end_page": end_page, "page_list_size": page_list_size, "total_page": total_page,  "prev_list": prev_list, "next_list": next_list, "links": links})

def write(request):
    return render(request, 'write.html')

def insert(request):
    fname=''
    fsize=0
    if 'file' in request.FILES:
        file = request.FILES['file']
        fname = file._name

        with open("%s%s" % (UPLOAD_DIR, fname), 'wb') as fp:
            for chunk in file.chunks():
                fp.write(chunk)
        fsize = os.path.getsize(UPLOAD_DIR+fname)
    row = Board(writer=request.POST['writer'], title=request.POST['title'],
                content=request.POST['content'], filename=fname, filesize=fsize)
    row.save()
    return redirect('/')

def detail(request):
    # 조회수 증가처리
    id = request.GET['idx']
    row = Board.objects.get(idx=id)
    row.hit_up()
    row.save()

    commentList = Comment.objects.filter(board_idx=id).order_by('-idx')

    filesize = '%.2f' % (row.filesize / 1024)
    return render(request, 'detail.html',  {"row": row, "filesize": filesize, "commentList": commentList})

def update(request):
    id = request.POST['idx']
    row_src = Board.objects.get(idx=id)

    fname = row_src.filename  # 수정 전의 첨부파일 이름
    fsize = row_src.filesize  # 수정 전의 첨부파일 크기
    hit = row_src.hit  # 수정 전의 조회수
    down = row_src.down  # 수정 전의 다운로드 횟수
    if "file" in request.FILES:
        file = request.FILES["file"]
        fname = file._name

        with open("%s%s" % (UPLOAD_DIR, fname), "wb") as fp:
            for chunk in file.chunks():
                fp.write(chunk)

        fsize = os.path.getsize(UPLOAD_DIR + fname)

    row_new = Board(idx=id, writer=request.POST["writer"], title=request.POST["title"], content=request.POST["content"],
                    filename=fname, filesize=fsize, hit=hit, down=down)
    row_new.save()
    return redirect("/")

def delete(request):
    id = request.POST["idx"]
    Board.objects.get(idx=id).delete()
    return redirect("/")

def download(request):
    id = request.GET['idx']
    row = Board.objects.get(idx=id)
    path = UPLOAD_DIR + row.filename

    filename = os.path.basename(path) #디렉토리를 제외한 파일이름
    # filename = filename.encode("utf-8")
    filename = quote(filename) # 한글,특수문자 인코딩 처리

    with open(path, 'rb') as file:
        response = HttpResponse(file.read(), content_type="application/octet-stream")
        # content_type: 다양한 종류의 파일
        # UTF-8 뒤에 작은 따옴표 2개
        response["Content-Disposition"] = "attachment; filename*=UTF-8''{0}".format(filename)
        row.down_up()
        row.save()
        return response

def reply_insert(request):
    id = request.POST["idx"]
    row = Comment(board_idx=id, writer=request.POST["writer"], content=request.POST["content"])

    row.save()
    # redirect 함수 대신 HttpResponseRedirect 함수를 사용해야 함
    return HttpResponseRedirect("detail?idx=" + id)


```

static 파일 저장을 위한 디렉토리를 미리 생성해야 함

board/static/images/file.png (첨부파일표시용)

#### board/templates/list.html
```board/templates/list.html
<!DOCTYPE html>
<html>
<head>
</head>
<body>
    <h2>게시판</h2>
    <form method="post" >
      {% csrf_token %}
        <select name="search_option">
          {% if search_option == "writer" %}
            <option value="writer" selected>이름</option>
            <option value="title">제목</option>
            <option value="content">내용</option>
            <option value="all">이름+제목+내용</option>
          {% elif search_option == "title" %}
            <option value="writer">이름</option>
            <option value="title" selected>제목</option>
            <option value="content">내용</option>
            <option value="all">이름+제목+내용</option>
          {% elif search_option == "content" %}
            <option value="writer">이름</option>
            <option value="title">제목</option>
            <option value="content" selected>내용</option>
            <option value="all">이름+제목+내용</option>
          {% else %}
            <option value="writer">이름</option>
            <option value="title">제목</option>
            <option value="content">내용</option>
            <option value="all" selected>이름+제목+내용</option>
          {% endif %}
        </select>
        <input name="search" value="{{search}}">
        <input type="submit" value="검색">
    </form>

    <a href="write">글쓰기</a>
    <table border="1">
        <tr>
            <th>번호</th>
            <th>이름</th>
            <th>제목</th>
            <th>날짜</th>
            <th>조회수</th>
            <th>첨부파일</th>
            <th>다운로드</th>
        </tr>
        {% for row in boardList %}
        <tr align="center">
            <td>{{row.idx}}</td>
            <td>{{row.writer}}</td>
            <td><a href="detail?idx={{row.idx}}">{{row.title}}</a></td>
            <td>{{row.post_date}}</td>
            <td>{{row.hit}}</td>
            <td>
                {% load static %}
                {% static "" as baseUrl %}
                {% if row.filesize > 0 %}
                <a href="download?idx={{row.idx}}"><img src="{{baseUrl}}images/fileicon2.png" width="30px" height="30px"></a>
                {% endif %}
            </td>
            <td>{{row.down}}</td>
        </tr>
        {% endfor %}
<!-- 페이지 네비게이션 -->
        <tr>
            <td colspan="7" align="center">
{% if start_page >= page_list_size %}
    <a href="?start={{prev_list}}">[이전]</a>
{% endif %}
<!-- 페이지 리스트 출력 -->
{% autoescape off %} <!-- html 코드가 그대로 보이지 않도록 처리 -->
{% for link in links %}
    {{link}}
{% endfor %}
{% endautoescape %}
<!-- 다음 페이지 리스트 출력 -->
{% if total_page > end_page %}
    <a href="?start={{next_list}}">[다음]</a>
{% endif %}
            </td>
        </tr>
    </table>
</body>
</html>


```


#### board/templates/detail.html
```board/templates/detail.html
<!DOCTYPE html>
<html>
<head>
<script>
function home(){
    location.href="/"
}
function update(){
    document.form1.action="update"
    document.form1.submit();
}
function del(){
    document.form1.action="delete"
    document.form1.submit();
}
</script>
</head>
<body>

    <form method="post" name="form1" enctype="multipart/form-data">
      {% csrf_token %}
        <table border="1" width="700px">
            <tr>
                <td>조회수</td>
                <td>{{row.hit}}</td>
            </tr>
            <tr>
                <td> 이름</td>
                <td><input name="writer" value="{{row.writer}}"></td>
            </tr>
            <tr>
                <td> 제목</td>
                <td><input name="title" value="{{row.title}}"></td></tr>
            <tr>
                <td> 날짜</td>
                <td>{{row.post_date}}</td>
            </tr>
            <tr>
                <td> 내용</td>
                <td><textarea name="content" rows="5" cols="60">{{row.content}}</textarea></td>
            </tr>
            <tr>
                <td> 첨부파일</td>
                <td>
                    {% if row.filesize > 0 %}
                        <a href="download?idx={{row.idx}}">{{row.filename}}</a>
                        ( {{filesize}}KB )
                        <br>
                    {% endif %}
                    <input type="file" name="file">
                </td>
            </tr>
            <tr>
                <td colspan="2" align="center">
                    <input type="hidden" name="idx" value="{{row.idx}}">
                    <input type="button" value="목록" onclick="home()">
                    <input type="button" value="수정" onclick="update()">
                    <input type="button" value="삭제" onclick="del()">
                </td>
            </tr>
        </table>
    </form>
    <!-- 댓글 작성 -->
    <form method="post" action="reply_insert">
      {% csrf_token %}
      <input name="writer" placeholder="이름"><br>
      <textarea rows="5" cols="80" name="content"
      placeholder=" 댓글을 작성하세요"></textarea><br>
      <input type="hidden" name="idx" value="{{row.idx}}">
      <button> 댓글쓰기</button>
    </form>
    <!-- 댓글 목록을 출력할 영역 -->
    <table border="1" width="700px">
        {% for row in commentList %}
        <tr>
           <td>
               {{row.writer}} ( {{row.post_date}} )<br>
                {{row.content}}
            </td>
        </tr>
        {% endfor %}
    </table>
</body>
</html>

```

#### board/templates/write.html
``` board/templates/write.html
<!DOCTYPE html>
<html>
<head>
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
<title>Insert title here</title>
</head>
<body>
<h2>글쓰기</h2>
<form method="post" action="insert" enctype="multipart/form-data">
  {% csrf_token %}
  <p>
   이름 <input name="writer" size="80"
       placeholder=" 이름을 입력하세요">
  </p>
  <p>
    제목 <input name="title" size="80"
       placeholder=" 제목을 입력하세요">
  </p>
  <p>
    <textarea name="content"
rows="3" cols="70" placeholder="내용을 입력하세요"></textarea>
  </p>
  <p>
    첨부파일
    <input type="file" name="file">
  </p>
  <p>
    <button>확인</button>
  </p>
</form>
</body>
</html>


```